In [3]:
import numpy as np  
import os
import pickle
import cv2
from tensorflow import keras
from keras import applications
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image
from keras.models import Model

In [4]:
model = keras.applications.vgg19.VGG19()

In [5]:
model = Model(inputs=model.input, outputs=model.get_layer('fc1').output)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [7]:
loaded_model1 = pickle.load(open(f'models/model1_svm.sav', 'rb'))
loaded_model2 = pickle.load(open(f'models/model2_svm.sav', 'rb'))

In [8]:
cap = cv2.VideoCapture(f'test_video/no_ball.avi')

In [9]:
count = 0
bufferCount = 0
globalWideVideo = []
globalOutVideo = []
globalSixVideo = []
globalNoBallVideo = []
globalNoActionVideo = []
buffer = []

In [10]:
th = 5
buff_th = 250
globalWideCounter = 0
globalOutCounter = 0
globalSixCounter = 0
globalNoBallCounter = 0
globalNoActionCounter = 0

In [13]:
while (True):
    ret, frame =cap.read()
    
    if ret:
        bufferCount = bufferCount + 1
        buffer.append(frame)
        height, width, layers = frame.shape
        size = (width, height)
        count = count + 1
        print ('success')
        frame = cv2.resize(frame, (224, 224))
        x = image.img_to_array(frame)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        #Feature Extraction Step
        features = model.predict(x) 
        
        #Classification Step
        predicted_values = loaded_model1.predict(features.reshape(1, -1)) 
        if predicted_values==2:
            predicted_values_2 = loaded_model2.predict(features.reshape(1, -1))
            choices = {'1':'noball', '2':'out', '3':'six', '4':'wide', '5':'noaction'}
            result = choices.get(np.str(int(predicted_values_2)), 'default')
            if result == 'noball':
                globalNoBallCounter = globalNoBallCounter + 1
                print('noball:')
            if result == 'out':
                globalOutCounter = globalOutCounter + 1
                print('out:')
            if result == 'six':
                globalSixCounter = globalSixCounter + 1
                print('six:')
            if result == 'wide':
                globalWideCounter = globalWideCounter + 1
                print('wide:')
            if result == 'noaction':
                globalNoActionCounter = globalNoActionCounter + 1
                print('noaction:')
            
    else:
        break
    ## Frame accumulation for buffer
    if bufferCount == buff_th:
        actionCount = {'noball': globalNoBallCounter, 'out': globalOutCounter,
                       'six': globalSixCounter, 'wide': globalWideCounter}

        winner = max(actionCount, key=actionCount.get)
        if winner == 'noball' and globalNoBallCounter >th:
            globalNoBallVideo.append(buffer)
        if winner == 'out'and globalOutCounter >th:
            globalOutVideo.append(buffer) 
        if winner == 'six'and globalSixCounter >th:
            globalSixVideo.append(buffer)
        if winner == 'wide' and globalWideCounter >th:
            globalWideVideo.append(buffer)
       
        bufferCount = 0
        globalWideCounter = 0
        globalOutCounter = 0
        globalSixCounter = 0
        globalNoBallCounter = 0
        globalNoActionCounter = 0
        buffer = []
    
actionCount = {'noball': globalNoBallCounter, 'out': globalOutCounter,
               'six': globalSixCounter, 'wide': globalWideCounter}
winner = max(actionCount, key=actionCount.get)
if winner == 'noball' and globalNoBallCounter >th:
    globalNoBallVideo.append(buffer)
if winner == 'out'and globalOutCounter >th:
    globalOutVideo.append(buffer) 
if winner == 'six'and globalSixCounter >th:
    globalSixVideo.append(buffer)
if winner == 'wide' and globalWideCounter >th:
    globalWideVideo.append(buffer)
        
cv2.destroyAllWindows()

print ('Summarizing Video...')

if globalNoBallVideo!=[]:
    noBallVideo = cv2.VideoWriter(f'no_ball1.avi', cv2.VideoWriter_fourcc(*'DIVX'),
                                  25, size)
    for i in range(len(globalNoBallVideo)):
        for j in range(len(globalNoBallVideo[i])):
            # writing to a image array
            noBallVideo.write(globalNoBallVideo[i][j])
    noBallVideo.release()

if globalOutVideo!=[]:
    outVideo = cv2.VideoWriter(f'out.avi', cv2.VideoWriter_fourcc(*'DIVX'),
                               25, size)
    for i in range(len(globalOutVideo)):
        for j in range(len(globalOutVideo[i])):
            # writing to a image array
            outVideo.write(globalOutVideo[i][j])
    outVideo.release()    

if globalWideVideo!=[]:
    wideVideo = cv2.VideoWriter(f'wide.avi', cv2.VideoWriter_fourcc(*'DIVX'),
                                25, size)
    for i in range(len(globalWideVideo)):
        for j in range(len(globalWideVideo[i])):
            # writing to a image array
            wideVideo.write(globalWideVideo[i][j])
    wideVideo.release()

if globalSixVideo!=[]:
    sixVideo = cv2.VideoWriter(f'sixes.avi', cv2.VideoWriter_fourcc(*'DIVX'),
                               25, size)
    for i in range(len(globalSixVideo)):
        for j in range(len(globalSixVideo[i])):
            # writing to a image array
            sixVideo.write(globalSixVideo[i][j])
    sixVideo.release()

success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
six:
success
six:
success
noaction:
success
noaction:
success
noaction:
success
noaction:
success
noaction:
success
noaction:
success
noaction:
success
success
noaction:
success
noaction:
success
success
success
success
six:
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success